In [ ]:
import pandas as pd
import json
import os

# Read the Excel file into a DataFrame
excel_file = '../inputdata/mapping.xlsx'
sheet_name_sources = 'Sources'
sheet_name_targets = 'Target'
df_sources = pd.read_excel(excel_file, sheet_name=sheet_name_sources)
df_targets = pd.read_excel(excel_file, sheet_name=sheet_name_targets)

# Convert the table DataFrame to a list of dictionaries
json_data_sources = df_sources.to_json(orient='records')
json_data_targets = df_targets.to_json(orient='records')
prompt = "Can you create a big query sql that uses the sources and maps to the target definition. please ensure left joins to the main identifier table \n Json for the Sources =" + json_data_sources + " \n Json for the Targets = " + json_data_targets

In [ ]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv('GEMINI_API_KEY')

import os
import requests


# Set the API endpoint and request body
url = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent'
payload = {
    "contents": [
        {
            "parts": [
                {
                    "text": "Explain how AI works"
                }
            ]
        }
    ]
}

# Make the API request
headers = {
    'Content-Type': 'application/json'
}
response = requests.post(url, headers=headers, json=payload, params={'key': api_key})

# Check the response
if response.status_code == 200:
    print(response.json())
else:
    print(f'Error: {response.status_code} - {response.text}')
